In [14]:
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
from stats import RandomSampling, MetropolisHastingsStep, MetropolisHastings
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import erfinv

# Stats

## Sampling a Probability Distribution

### Sampling from Random Points

To sample a distribution that with a maximum value of h and which exists over some finite range the are several methods. A very simple approach is to sample our probability distribution by choosing random points. From here we either accept a point if it lies under our probability distribution curve, or we reject it if it lies below the curve.

Below you can define a probability distribution that we will sample using the technique described above. Try drawing a box around your function and then seeing how many tries it takes to get a point that lies inside you distribution. Make sure to have your function be completely encased by the rectangle and that your function is positive.

In [15]:
def prob_func(x):
	return 1/np.sqrt(2*np.pi)*np.exp(-(x**2)/2)

In [ ]:
out = widgets.Output()

w_slider = widgets.FloatText(value = 7, description="Width")
h_slider = widgets.FloatText(value = 0.45, description = "Height")
c_slider = widgets.FloatText(value = 0, description = "Center")

button = widgets.Button(description="Sample New Point")

def draw_distribution(*args):
	with out:
		out.clear_output(wait=True)
		width = max(w_slider.value,0.1)
		height = max(h_slider.value,0.1)
		center = c_slider.value
		plt.figure(figsize=(4, 4))
		plt.gca().add_patch(
			plt.Rectangle((center-width/2, 0), width, height, fill=False)
		)
		plt.xlim(center-width/2-width*0.2,center+width/2+width*0.2)
		plt.ylim(0,height+height*0.2)
		x_vals = np.linspace(center-width/2-width*0.2,center+width/2+width*0.2,100)
		y_vals = prob_func(x_vals)
		plt.plot(x_vals,y_vals)
		x,y,iters = RandomSampling(prob_func,(center-width/2,center+width/2),(0,height))
		if not np.isnan(x):
			plt.plot(x,y,'o')
		plt.show()
		print("Number of points discarded: ",iters)
		if not np.isnan(x):
			print("Random value: ", x)
		else:
			print("No random number found. Make sure the rectangle nicely surrounds your distribution.")

w_slider.observe(draw_distribution, 'value')
h_slider.observe(draw_distribution, 'value')
c_slider.observe(draw_distribution, 'value')
button.on_click(draw_distribution)

out.layout.height = '500px'
controls = widgets.VBox([w_slider,h_slider,c_slider,button])
view = widgets.HBox([controls,out])


draw_distribution()
display(view)


Now that our region has been defined, what we will do is sample many points in this region and compare the resulting distribution to the one that we are trying sample from.

In [17]:
out2 = widgets.Output()
n_widget = widgets.BoundedIntText(value=300,min=1,max=10000, description = "Sample Number")
b_widget = widgets.BoundedIntText(value=20,min=5,max=100,description="Number of Bins")
sample_button = widgets.Button(description = "Re-sample")

def sample_distribution(*args):
	with out2:
		out2.clear_output(wait=True)
		width = w_slider.value
		height = h_slider.value
		center = c_slider.value
		n = n_widget.value
		x_values = np.zeros(n)
		iterations = np.zeros(n)
		for i in range(n):
			x,y,iters = RandomSampling(prob_func,(center-width/2,center+width/2),(0,height))
			x_values[i] = x
			iterations[i] = iters
		plt.hist(x_values,bins = b_widget.value,density=True)
		plt.gca().add_patch(
			plt.Rectangle((center-width/2, 0), width, height, fill=False)
		)
		x_vals = np.linspace(center-width/2-width*0.2,center+width/2+width*0.2,100)
		y_vals = prob_func(x_vals)
		plt.plot(x_vals,y_vals)
		plt.show()
		print("Total Extra Points Sampled: ", np.sum(iterations))


sample_button.on_click(sample_distribution)

sample_distribution()

controls2 = widgets.VBox([n_widget,b_widget,sample_button])
view2 = widgets.HBox([controls2,out2])


display(view2)

The one drawback to this method is that if your function takes up only a small fraction of the area of the uniform cover, the the number of extra points that need to be sampled also grows a lot. To see this you can try experimenting by making the box much bigger than your distribution function, then compare the total number of extra points that need to be sampled to the number of points requested.

### Sampling using Metropolis Hastings

As mentioned in the previous section, the one drawback to the sampling method above is the wasted computation that comes from having to discard points. So we now turn to an approach that will hopefully eliminate the need to resample points. The name of the algorithm is Metropolis-Hastings and it works by basically occupying a point on the probability distribution and then moving around the distribution based on the probability of the distribution function we are trying to sample.

Let's start with some probability density function that we would like to generate random numbers for. We will denote this function as:

$P(x)$

In order for us to sample this we must already have a distribution that we can obtain random numbers from. We will call this known distribution:

$f(x)$

What we will do is begin by picking some random point $x_0$ that lies within our desired distribution. One assumption that is made in this write up is that $f(x)$ is symmetric. If you so desire to know how this algorithm works for non-symmetric $f(x)$, then there is a great [Wikipedia Page](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) that has a description on how to implement the algorithm for non-symmetric $f(x)$.

Now we have some $x_0$ which is our first random point. What we want is to sample n random points. So we need a procedure to get from $x_t$ to $x_{t+1}$.
If our distribution is centered at 0 and with some step size denoted by $dx$, then what we will do is:

1. Generate a random number from $f(x)$ which we will call $x_f$
2. We multiply by the step size and add our current $x_t$ value. $x'=x_t + x_f*dx$
3. We now have two candidates for $x_{t+1}$; $x_t$ and $x'$. To help us decide which value to pick we will compute the following value $a = \frac{P(x')}{P(x_t)}$
4. Now if the probability density at $x'$ is greater than the probability density at our current location $x_t$, i.e. $a \geq 1$ we will immediately accept $x'$ as $x_{t+1}$
5. If the probability density at $x'$ is less than the probability density at our current location $x_t$, i.e. $a < 1$ then we will accept $x'$ with probability $a$ and $x_t$ with probability $1-a$
6. To do this we generate a uniform random number $u$ and if $u \leq a$ then $x_{t+a} = x'$ otherwise $x_{t+1} = x_t$.

To get n random numbers sampled from $P(x)$, we simply continue this procedure starting at $x_0$ until we get $x_{n-1}$.


In the following sections, the known probability distribution that will be used is the uniform distribution from -1 to 1. In this way dx will basically correspond to how fast we will be able to explore the distribution. For small values of dx, it will take a large value of n to sample the entire distribution, while with large dx values, the sampling will become more eratic, but still converge if n is large enough.

You can set the bounds of your distribution function below.

In [18]:
xmin, xmax = -4,4

Now we will be able to watch the distribution get sampled.

In [19]:
out3 = widgets.Output()

ip_widget = widgets.FloatText(value = (xmax-xmin)/2 + xmin, description = "Intial Position")
dx_widget = widgets.FloatText(value = 1, description = "dx :")

step_button = widgets.Button(description = "Take Step")

xs = [ip_widget.value]

def take_step(*args):
	with out3:
		out3.clear_output(wait=True)
		x_next = MetropolisHastingsStep(prob_func,xs[-1],dx_widget.value)
		x_vals = np.linspace(xmin,xmax,100)
		y_vals = prob_func(x_vals)
		plt.plot(x_vals,y_vals)
		plt.plot(xs,prob_func(np.asarray(xs)),'o-',alpha = 0.2)
		plt.plot(x_next,prob_func(x_next),'ro')
		plt.show()
		xs.append(x_next)

def reset_xs(*args):
	xs.clear()
	xs.append(ip_widget.value)
	take_step(args)

take_step()

ip_widget.observe(reset_xs, 'value')
step_button.on_click(take_step)


display(ip_widget,dx_widget,step_button,out3)

FloatText(value=0.0, description='Intial Position')

FloatText(value=1.0, description='dx :')

Button(description='Take Step', style=ButtonStyle())

Output()

Below we will see how the number of points we pick changes the sampling of the distribution. The more points, the closer the resemblance to the desired distribution.

In [9]:
out4 = widgets.Output()
n_widget = widgets.BoundedIntText(value=300,min=1,max=10000, description = "Sample Number")
b_widget = widgets.BoundedIntText(value=20,min=5,max=100,description="Number of Bins")
sample_button = widgets.Button(description = "Re-sample")

def sample_distribution(*args):
	with out4:
		out4.clear_output(wait=True)
		width = w_slider.value
		height = h_slider.value
		center = c_slider.value
		n = n_widget.value
		x_values = MetropolisHastings(prob_func,ip_widget.value,dx_widget.value,n_widget.value)
		x_vals = np.linspace(center-width/2-width*0.2,center+width/2+width*0.2,100)
		y_vals = prob_func(x_vals)
		plt.hist(x_values,bins = b_widget.value,density=True)
		plt.plot(x_vals,y_vals)
		plt.show()


sample_button.on_click(sample_distribution)

sample_distribution()

controls2 = widgets.VBox([n_widget,b_widget,sample_button])
view2 = widgets.HBox([controls2,out4])


display(view2)

### Inverse Transform Sampling

If we want to generate random numbers for a distribution $P(x)$ given we know the cumulative distribution function (CDF) then we can do so using a uniform random variable in the range [0,1). The CDF should be a monotonically increasing and take values between 0 and 1: we will call this function $F(x)$. If we assume that $F(x)$ is invertable, then we can denote this function as $F^{-1}(x)$. This way we can go from our random value between 0 and 1 to a random number in our distribution of interest.

For the probability distribution used in this demo:

$\frac{1}{\sqrt{2\pi}} e^{-\frac{x^2}{2}}$

The CDF is as follows:

$F(x) = \frac{1}{2}\left( 1 + \mathrm{Erf} \left(\frac{x}{\sqrt{2}}\right)\right)$

Where Erf is the error function. There is no closed form solution to the error function which makes it not so ideal to use for obtaining the inverse of, but there are libraries such as scipy which can compute it. We will now have to solve the above equation for x.

$ x = \sqrt{2}\mathrm{Erf}^{-1}(2F(x)-1)$

If we now take our random variable to be the result of F(x) we can now get a corresponding variable from our desired probability ditribution by obtaining x from our random number.

In [20]:
def inv_cum_prob_dist(x):
    return np.sqrt(2)*erfinv(2*x-1)

In [21]:
out = widgets.Output()
n_widget = widgets.BoundedIntText(value=300,min=1,max=10000, description = "Sample Number")
b_widget = widgets.BoundedIntText(value=20,min=5,max=100,description="Number of Bins")
sample_button = widgets.Button(description = "Re-sample")

def sample_distribution_icdf(*args):
	with out:
		out.clear_output(wait=True)
		width = w_slider.value
		height = h_slider.value
		center = c_slider.value
		n = n_widget.value
		x_values = inv_cum_prob_dist(np.random.rand(n))
		plt.hist(x_values,bins = b_widget.value,density=True)
		x_vals = np.linspace(center-width/2-width*0.2,center+width/2+width*0.2,100)
		y_vals = prob_func(x_vals)
		plt.plot(x_vals,y_vals)
		plt.show()


sample_button.on_click(sample_distribution_icdf)

sample_distribution_icdf()

controls2 = widgets.VBox([n_widget,b_widget,sample_button])
view2 = widgets.HBox([controls2,out])

display(view2)

## Sampling Random Points from the Surface of a Sphere

If we were to take the naive approach of sampling the surface of a sphere by picking a uniform random variable of the two spherical angles $\phi$ and $\theta$ one would quickly notice that this distribution is in fact not uniformly distributed on the surface of the sphere.

In [22]:

out6 = widgets.Output()
n_widget = widgets.BoundedIntText(value=300,min=1,max=10000, description = "Sample Number")
sample_button = widgets.Button(description = "Re-sample")

def sphere_plot(*args):
	with out6:
		%matplotlib widget
		plt.close('all')
		out6.clear_output(wait=True)
		thetas = np.random.rand(n_widget.value) * np.pi
		phis = np.random.rand(n_widget.value) * 2*np.pi

		xs = np.cos(phis)*np.sin(thetas)
		ys = np.sin(phis)*np.sin(thetas)
		zs = np.cos(thetas)

		r = 0.95
		u = np.linspace(0, 2*np.pi, 100)
		v = np.linspace(0, np.pi, 100)

		# Parametric equations for a sphere
		x = r * np.outer(np.cos(u), np.sin(v))
		y = r * np.outer(np.sin(u), np.sin(v))
		z = r * np.outer(np.ones_like(u), np.cos(v))

		# Create 3D axes
		fig = plt.figure()
		ax = fig.add_subplot(111, projection='3d')

		# Plot the surface
		ax.plot_surface(x, y, z, color='cornflowerblue', alpha=0.5)
		ax.scatter(xs,ys,zs,color="red",marker='.')

		# Labels
		ax.set_xlabel('X')
		ax.set_ylabel('Y')
		ax.set_zlabel('Z')

		# Important: equal aspect ratio
		ax.set_box_aspect([1, 1, 1])

		plt.show()
		%matplotlib inline

sample_button.on_click(sphere_plot)
sphere_plot()

display(n_widget,sample_button,out6)




BoundedIntText(value=300, description='Sample Number', max=10000, min=1)

Button(description='Re-sample', style=ButtonStyle())

Output()

If you sample enough points and look at the sphere top down, you will notice that points tend to cluster close to the poles of the sphere. To fix this we must sample using the following scheme:

- Get two random variables $v$ and $u$ in the range 0 to 1.
- Asuming $\theta$ measures the angle from the z-axiz and $\phi$ measures the angle in the xy-plane from the x axis to our point, then we must correct the $\theta$ points so that they are less clustered around the center. To do this, we simply obtain our variables in the following way:

$\theta = \cos^{-1}(2u-1)$

$\phi = 2\pi v$

In [23]:
out7 = widgets.Output()
n_widget = widgets.BoundedIntText(value=300,min=1,max=10000, description = "Sample Number")
sample_button = widgets.Button(description = "Re-sample")

def sphere_plot_corrected(*args):
	with out7:
		%matplotlib widget
		plt.close('all')
		out7.clear_output(wait=True)
		thetas = np.arccos(2*np.random.rand(n_widget.value)-1)
		phis = np.random.rand(n_widget.value) * 2*np.pi

		xs = np.cos(phis)*np.sin(thetas)
		ys = np.sin(phis)*np.sin(thetas)
		zs = np.cos(thetas)

		r = 0.95
		u = np.linspace(0, 2*np.pi, 100)
		v = np.linspace(0, np.pi, 100)

		# Parametric equations for a sphere
		x = r * np.outer(np.cos(u), np.sin(v))
		y = r * np.outer(np.sin(u), np.sin(v))
		z = r * np.outer(np.ones_like(u), np.cos(v))

		# Create 3D axes
		fig = plt.figure()
		ax = fig.add_subplot(111, projection='3d')

		# Plot the surface
		ax.plot_surface(x, y, z, color='cornflowerblue', alpha=0.5)
		ax.scatter(xs,ys,zs,color="red",marker='.')

		# Labels
		ax.set_xlabel('X')
		ax.set_ylabel('Y')
		ax.set_zlabel('Z')

		# Important: equal aspect ratio
		ax.set_box_aspect([1, 1, 1])

		plt.show()
		%matplotlib inline

sample_button.on_click(sphere_plot_corrected)
sphere_plot_corrected()

display(n_widget,sample_button,out7)

BoundedIntText(value=300, description='Sample Number', max=10000, min=1)

Button(description='Re-sample', style=ButtonStyle())

Output()